In [1]:
import pandas as pd
import numpy as np

## Create our dataset

In [79]:
def get_dataset(size):
    # Create Fake Dataset
    df = pd.DataFrame()
    df['size'] = np.random.choice(['big','medium','small'], size)
    df['age'] = np.random.randint(1, 50, size)
    df['team'] = np.random.choice(['red','blue','yellow','green'], size)
    df['win'] = np.random.choice(['yes','no'], size)
    dates = pd.date_range('2020-01-01', '2022-12-31')
    df['date'] = np.random.choice(dates, size)
    df['prob'] = np.random.uniform(0, 1, size)
    return df

def set_dtypes(df):
    df['size'] = df['size'].astype('category')
    df['team'] = df['team'].astype('category')
    df['age'] = df['age'].astype('int16')
    df['win'] = df['win'].map({'yes':True, 'no': False})
    df['prob'] = df['prob'].astype('float32')
    return df


## CSV
- 46 MB
- 8.8s to save
- 0.5s to read

In [26]:
%%timeit
df = get_dataset(1_000_000)
df.to_csv('test_csv.csv', index=True)

8.82 s ± 22.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
df = pd.read_csv('test_csv.csv', index_col=[0])

562 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
!ls -GFlash test_csv.csv

53M -rw-rw-r-- 1 robmulla 53M Apr  2 14:12 test_csv.csv


In [31]:
df = get_dataset(1_000_000)
df = set_dtypes(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   size    1000000 non-null  category      
 1   age     1000000 non-null  int16         
 2   team    1000000 non-null  category      
 3   win     1000000 non-null  bool          
 4   date    1000000 non-null  datetime64[ns]
 5   prob    1000000 non-null  float16       
dtypes: bool(1), category(2), datetime64[ns](1), float16(1), int16(1)
memory usage: 14.3 MB


In [32]:
df.to_csv('test_csv.csv', index=False)
df = pd.read_csv('test_csv.csv')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   size    1000000 non-null  object 
 1   age     1000000 non-null  int64  
 2   team    1000000 non-null  object 
 3   win     1000000 non-null  bool   
 4   date    1000000 non-null  object 
 5   prob    1000000 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(3)
memory usage: 39.1+ MB


In [46]:
df = pd.read_csv('test_csv.csv',
                 dtype={'size':'category',
                       'int64':'int16',
                       'team':'category',})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype   
---  ------  --------------    -----   
 0   size    1000000 non-null  category
 1   age     1000000 non-null  int64   
 2   team    1000000 non-null  category
 3   win     1000000 non-null  bool    
 4   date    1000000 non-null  object  
 5   prob    1000000 non-null  float64 
dtypes: bool(1), category(2), float64(1), int64(1), object(1)
memory usage: 25.7+ MB


# Pickle
- 0.8s to write
- 0.3s to read
- 43MB

In [47]:
df = get_dataset(1_000_000)
%timeit df.to_pickle('test.pickle')
%timeit df_pickle = pd.read_pickle('test.pickle')

822 ms ± 9.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
302 ms ± 2.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
!ls -GFlash test.pickle

43M -rw-rw-r-- 1 robmulla 43M Apr  2 14:22 test.pickle


In [49]:
df = get_dataset(1_000_000)
df = set_dtypes(df)
df.to_pickle('test.pickle')
df_pickle = pd.read_pickle('test.pickle')

In [51]:
df_pickle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   size    1000000 non-null  category      
 1   age     1000000 non-null  int16         
 2   team    1000000 non-null  category      
 3   win     1000000 non-null  bool          
 4   date    1000000 non-null  datetime64[ns]
 5   prob    1000000 non-null  float16       
dtypes: bool(1), category(2), datetime64[ns](1), float16(1), int16(1)
memory usage: 14.3 MB


# Parquet
```
!pip install pyarrow
!pip install fastparquet
```

- 0.3ms to write
- 0.08ms to read
- 11MB

In [52]:
df = get_dataset(1_000_000)
%timeit df.to_parquet('test.parquet')
%timeit df_parquet = pd.read_parquet('test.parquet')

308 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
82.3 ms ± 3.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [53]:
!ls -GFlash test.parquet

11M -rw-rw-r-- 1 robmulla 11M Apr  2 14:25 test.parquet


In [56]:
# Read in specific columns
df_example = pd.read_parquet('test.parquet', columns=['date','win'])

# Feather
- 0.22s write
- 0.075s read
- 29MB

In [57]:
df = get_dataset(1_000_000)
%timeit df.to_feather('test.feather')
%timeit df_feather = pd.read_feather('test.feather')

220 ms ± 3.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
75.7 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [60]:
!ls -GFlash test.feather

29M -rw-rw-r-- 1 robmulla 29M Apr  2 14:28 test.feather


# CSV vs Pickle vs Parquet vs Feather

In [70]:
print('Reading and writing CSV')
df = get_dataset(5_000_000)
df = set_dtypes(df)
%time df.to_csv('test.csv')
%time df_csv = pd.read_csv('test.csv')

Reading and writing CSV
CPU times: user 39 s, sys: 254 ms, total: 39.2 s
Wall time: 39.3 s
CPU times: user 2.12 s, sys: 124 ms, total: 2.24 s
Wall time: 2.24 s


In [83]:
print('Reading and writing Pickle')
df = get_dataset(5_000_000)
df = set_dtypes(df)
%time df.to_pickle('test.pickle')
%time df_pickle = pd.read_pickle('test.pickle')

Reading and writing Pickle
CPU times: user 5.32 ms, sys: 64.2 ms, total: 69.5 ms
Wall time: 249 ms
CPU times: user 0 ns, sys: 30.2 ms, total: 30.2 ms
Wall time: 30.2 ms


In [84]:
print('Reading and writing Parquet')
df = get_dataset(5_000_000)
df = set_dtypes(df)
%time df.to_parquet('test.parquet')
%time df_parquet = pd.read_parquet('test.parquet')

Reading and writing Parquet
CPU times: user 472 ms, sys: 52.3 ms, total: 524 ms
Wall time: 512 ms
CPU times: user 280 ms, sys: 124 ms, total: 404 ms
Wall time: 129 ms


In [85]:
print('Reading and writing Feather')
df = get_dataset(5_000_000)
df = set_dtypes(df)
%time df.to_feather('test.feather')
%time df_feather = pd.read_feather('test.feather')

Reading and writing Feather
CPU times: user 308 ms, sys: 79.3 ms, total: 387 ms
Wall time: 307 ms
CPU times: user 142 ms, sys: 88.3 ms, total: 230 ms
Wall time: 102 ms


In [86]:
!ls -GFlash test.csv test.pickle test.parquet test.feather

215M -rw-rw-r-- 1 robmulla 215M Apr  2 14:41 test.csv
 49M -rw-rw-r-- 1 robmulla  49M Apr  2 14:45 test.feather
 33M -rw-rw-r-- 1 robmulla  33M Apr  2 14:45 test.parquet
 82M -rw-rw-r-- 1 robmulla  82M Apr  2 14:45 test.pickle
